In [6]:
import pydicom
import os 
import numpy as np
import pandas as pd
import cv2 
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import shutil

Vado a costruire le liste dei path partendo dai CSV

In [7]:
#ds = pd.read_csv("./pre_retrieve_100_accnum.csv")
#ds = pd.read_csv("/mnt/databaseRICERCA_SDN/ONWAY/PACS/metadata/nonbrain_151_accnum.csv",sep = ";")
ds = pd.read_csv("/mnt/databaseRICERCA_SDN/ONWAY/PACS/metadata/pre_retrieve_100_accnum.csv", sep =";")
true = ds.loc[ds["is_brain_manual"] == True]
false =ds.loc[ds["is_brain_manual"] == False].dropna()
true_path = [os.path.dirname(x) for x in list(true["dicom_images.0.file_path"])]
false_path = [os.path.dirname(x) for x in list(false["dicom_images.0.file_path"])]


def find_fetta_centrale(path):
    fetta_centrale = list()
    no_fetta_centrale = list()
    for i in path  :
        try :
            fetta_centrale.append(os.path.join(i,(os.listdir(i))[1]))
        except :
            no_fetta_centrale.append(i)
    return fetta_centrale,no_fetta_centrale
                                    
fetta_centrale_true,no_fetta_centrale_true = find_fetta_centrale(true_path)
fetta_centrale_false,no_fetta_centrale_false = find_fetta_centrale(false_path)

In [3]:
dc = pydicom.dcmread(fetta_centrale_true[0])
dc.SeriesInstanceUID                 

'1.3.46.670589.11.17524.5.0.8876.2012010206340442000'

Converto le immagini da dicom a png, ogni immagine è rinominato con il nome della serie

In [ ]:
def dicom_png(path_image,path_save):
    dc = pydicom.dcmread(path_image)
    names  = os.path.join(path_save,(dc.SeriesInstanceUID + ".png"))
    plt.imsave(names, dc.pixel_array,cmap ="gray") 
path_save_true  = "/mnt/databaseRICERCA_SDN/ONWAY/PACS/Classificazione/true/"
path_save_false = "/mnt/databaseRICERCA_SDN/ONWAY/PACS/Classificazione/false/"

for path in tqdm(fetta_centrale_true) :
    dicom_png(path,path_save_true)
    
for path in tqdm(fetta_centrale_false) :
    dicom_png(path,path_save_false)

Separo le immagini, serve per provare ad usare i dicom direttamente senza andare ad effettuare la conversione in png

In [15]:
def divisore(path_image,path_save) : 
    dc = pydicom.dcmread(path_image)
    names  = os.path.join(path_save,(dc.SeriesInstanceUID + ".dcm"))
    shutil.copy(path_image, names)
#path_save_true  = "/mnt/databaseRICERCA_SDN/ONWAY/PACS/Classificazione/dcm/true/"
#path_save_false = "/mnt/databaseRICERCA_SDN/ONWAY/PACS/Classificazione/dcm/false/"
path_save_true  = "/home/marioverd/dcm/true/"
path_save_false = "/home/marioverd/dcm/false/"


for path in tqdm(fetta_centrale_true) :
    divisore(path,path_save_true)
    
for path in tqdm(fetta_centrale_false) :
    divisore(path,path_save_false)
    


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:01<00:00, 74.46it/s]


**Creo le immagini per il super-test** 



In [9]:
ds = pd.read_csv("/mnt/databaseRICERCA_SDN/ONWAY/PACS/metadata/pre_retrieve_151_accnum_20230210.csv")
path_test =  [os.path.dirname(x) for x in list(ds["dicom_images.0.file_path"])]
                                    
fetta_centrale_test,no_fetta_centrale_test = find_fetta_centrale(path_test)
path_save_test = "/home/marioverd/dcm/test/"
for path in tqdm(fetta_centrale_true) :
    divisore(path,path_save_test)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 758/758 [00:07<00:00, 100.38it/s]


Converto le super serie 


In [11]:
ds = pd.read_csv("./risultati/prova_super_test.csv")
true = ds.loc[ds["predizione"] == 1]["path"]
false =ds.loc[ds["predizione"] == 0]["path"]


def dicom_png(path_image,path_save):
    dc = pydicom.dcmread(path_image)
    names  = os.path.join(path_save,(dc.SeriesInstanceUID + ".png"))
    plt.imsave(names, dc.pixel_array,cmap ="gray") 
path_save_true  = "/mnt/databaseRICERCA_SDN/ONWAY/PACS/Classificazione/test_png/true/"
path_save_false = "/mnt/databaseRICERCA_SDN/ONWAY/PACS/Classificazione/test_png/false/"

for path in tqdm(fetta_centrale_true) :
    dicom_png(path,path_save_true)
    
for path in tqdm(fetta_centrale_false) :
    dicom_png(path,path_save_false)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:09<00:00, 13.11it/s]
